In [4]:
# import torch
# print(f"CUDA available: {torch.cuda.is_available()}")
# if torch.cuda.is_available():
#     print(f"CUDA version: {torch.version.cuda}")
#     print(f"Number of GPUs: {torch.cuda.device_count()}")
#     print(f"GPU Name: {torch.cuda.get_device_name(0)}")
# else:
#     print("CUDA is not available. Please check your installation.")


AttributeError: module 'torch' has no attribute 'cuda'

In [3]:
# import numpy as np
# import pandas as pd

# # Charger les données de validation
# val_data = pd.read_csv('data/processed/val/V3_feat_val.txt', sep=' ', header=None)

# # Vérifier la distribution des classes dans les labels de validation
# class_counts = val_data['Classification'].value_counts()
# print(f'Distribution des classes dans les données de validation:\n{class_counts}')


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/val/V3_feat_val.txt'

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/train/V3_feat_train.txt'

In [ ]:
classes = {'arch': 0,
           'columns': 1,
           'moldings': 2,
           'floor': 3,
           'door & window': 4,
           'wall': 5,
           'stairs': 6,
           'vault': 7,
           'roof': 8,
           'other': 9,
           }

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
import yaml
from data.custom_dataset import PointCloudDataset
from models.my_model import PointNet

import sys
import os

# Get the absolute path of the current notebook file
notebook_path = '/c:/N7/Stage_Bradford/notebooks/data_exploration.ipynb'

# Get the absolute path of the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_path, '..'))

# Append the parent directory to the sys.path
sys.path.append(parent_dir)

In [23]:
# Load dataset configuration
with open('configs/dataset_config.yaml', 'r') as file:
    dataset_config = yaml.safe_load(file)

# Load train dataset
train_dataset = PointCloudDataset(dataset_config['dataset']['train_path'])

# Extract labels from train dataset
train_labels = [label for _, label in train_dataset]

# Convert labels to a pandas DataFrame
train_labels_df = pd.DataFrame(train_labels, columns=['Label'])

# Plot the distribution of classes in the training dataset
plt.figure(figsize=(10, 6))
sns.countplot(data=train_labels_df, x='Label')
plt.title('Class Distribution in Training Data')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'configs/dataset_config.yaml'

In [8]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "3b3e6b78",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay\n",
    "import torch\n",
    "import yaml\n",
    "from data.custom_dataset import PointCloudDataset\n",
    "from models.my_model import PointNet\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "bfcf59d6",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load dataset configuration\n",
    "with open('configs/dataset_config.yaml', 'r') as file:\n",
    "    dataset_config = yaml.safe_load(file)\n",
    "\n",
    "# Load train dataset\n",
    "train_dataset = PointCloudDataset(dataset_config['dataset']['train_path'])\n",
    "\n",
    "# Extract labels from train dataset\n",
    "train_labels = [label for _, label in train_dataset]\n",
    "\n",
    "# Convert labels to a pandas DataFrame\n",
    "train_labels_df = pd.DataFrame(train_labels, columns=['Label'])\n",
    "\n",
    "# Plot the distribution of classes in the training dataset\n",
    "plt.figure(figsize=(10, 6))\n",
    "sns.countplot(data=train_labels_df, x='Label')\n",
    "plt.title('Class Distribution in Training Data')\n",
    "plt.xlabel('Class')\n",
    "plt.ylabel('Count')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "94e34876",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate the model and visualize the results\n",
    "def evaluate_model(model, dataloader):\n",
    "    model.eval()\n",
    "\n",
    "    all_labels = []\n",
    "    all_preds = []\n",
    "    with torch.no_grad():\n",
    "        for inputs, labels in dataloader:\n",
    "            outputs = model(inputs)\n",
    "            _, preds = torch.max(outputs, 1)\n",
    "            all_labels.extend(labels.cpu().numpy())\n",
    "            all_preds.extend(preds.cpu().numpy())\n",
    "\n",
    "    return all_labels, all_preds\n",
    "\n",
    "# Load model configuration\n",
    "with open('configs/model_config.yaml', 'r') as file:\n",
    "    model_config = yaml.safe_load(file)\n",
    "\n",
    "# Initialize and load the model\n",
    "model = PointNet(\n",
    "    model_config['model']['input_size'],\n",
    "    model_config['model']['hidden_layer1_size'],\n",
    "    model_config['model']['hidden_layer2_size'],\n",
    "    model_config['model']['num_classes']\n",
    ")\n",
    "model.load_state_dict(torch.load('checkpoints/best_model_augmented.pth'))\n",
    "\n",
    "# Load validation dataset\n",
    "val_dataset = PointCloudDataset(dataset_config['dataset']['val_path'])\n",
    "val_loader = DataLoader(val_dataset, batch_size=dataset_config['dataset']['batch_size'], shuffle=False)\n",
    "\n",
    "# Evaluate the model\n",
    "true_labels, predicted_labels = evaluate_model(model, val_loader)\n",
    "\n",
    "# Convert to pandas DataFrame\n",
    "results_df = pd.DataFrame({'True': true_labels, 'Predicted': predicted_labels})\n",
    "\n",
    "# Plot the confusion matrix\n",
    "cm = confusion_matrix(true_labels, predicted_labels)\n",
    "disp = ConfusionMatrixDisplay(confusion_matrix=cm)\n",
    "disp.plot(cmap='Blues')\n",
    "plt.title('Confusion Matrix')\n",
    "plt.show()\n",
    "\n",
    "# Save the results to a CSV file\n",
    "results_df.to_csv('results/evaluation_results.csv', index=False)\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined